## Przeprowadzenie operacji zmiany rozmiaru zdjęć potrzebnych do uczenia sieci

#### Importowanie bibliotek

In [1]:
import cv2

#### Funkcja `prepare_output_dirs`
Przygotowanie katalogó wyjściowych, jeśli nie istnieją.

In [2]:
def prepare_output_dirs(output_images_dir, output_masks_dir):
    output_images_dir.mkdir(parents=True, exist_ok=True)
    output_masks_dir.mkdir(parents=True, exist_ok=True)

#### Funkcja `resize_with_padding`
Funkcja do zmiany rozmiaru z zachowaniem proporcji

In [3]:
def resize_with_padding(image, size, is_mask = False):
    h, w = image.shape[:2]
    scale = min(size[1] / h, size[0] / w)
    new_w, new_h = int(w * scale), int(h * scale)
    interp = cv2.INTER_NEAREST if is_mask else cv2.INTER_AREA

    resized = cv2.resize(image, (new_w, new_h), interpolation=interp)
    top = (size[1] - new_h) // 2
    bottom = size[1] - new_h - top
    left = (size[0] - new_w) // 2
    right = size[0] - new_w - left

    border_val = 0 if is_mask else [0, 0, 0]
    return cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=border_val)

#### Funkcja `report_file_counts`
Wyświetlenie informacji na temat ilości zdjęć i masek

In [4]:
def report_file_counts(input_images_dir, input_masks_dir):
    image_paths = sorted(input_images_dir.iterdir())
    mask_paths = sorted(input_masks_dir.iterdir())

    print(f"Znaleziono {len(image_paths)} zdjęć i {len(mask_paths)} masek.")

#### Funkcja `process_images`
Funkcja odpowiedziala za przetworzenie zdjęć i ich zapis

In [5]:
def process_images(src_img_dir, src_mask_dir, dst_img_dir, dst_mask_dir, target_size):
    img_paths = sorted(src_img_dir.iterdir())

    for img_path in img_paths:
        base = img_path.stem
        mask_path = src_mask_dir / f"{base}.jpg"
        
        if not mask_path.exists():
            print(f"Brak maski dla {base}.")
            continue

        img = cv2.imread(str(img_path))
        msk = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        
        if img is None or msk is None:
            print(f"Błąd wczytywania {base}.")
            continue

        img_out = resize_with_padding(img, target_size, is_mask=False)
        msk_out = resize_with_padding(msk, target_size, is_mask=True)

        cv2.imwrite(str(dst_img_dir / f"{base}.jpg"), img_out)
        cv2.imwrite(str(dst_mask_dir / f"{base}.jpg"), msk_out)
    print("Zakończono przetwarzanie")

#### Funkcja `process_resize`
Funkcja wywołująca wszystkie wymagane funkcje do przeprowadzenia zmiany rozmiaru zdjęcia

In [6]:
def process_resize(input_images_dir, input_masks_dir, output_images_dir, output_masks_dir, target_size):
    prepare_output_dirs(output_images_dir, output_masks_dir)
    report_file_counts(input_images_dir, input_masks_dir)
   
    process_images(input_images_dir, input_masks_dir, output_images_dir, output_masks_dir, target_size)